In [1]:
import feltorutilities as fp
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2e' % x)

In [5]:
physicals=list()
for res in np.sort([3e-4,1e-4,3e-5,1e-5,3e-6,1e-6]):
    params = {"name" : "Compass",
    "beta" : 1e-4, "resistivity": res, #change both to change n_0
    "tau" : 1, "m_i" : fp.deuteron_mass, "R_0" : 545, "R": 0.545,
    "a": 0.175, "q":2, "scaleR" : 1.45, "Nz" : 32}
    physical = {"m_i" : fp.deuteron_mass,  "R": 0.545, "a": 0.175, "q":2, "scaleR" : 1.45, "Nz" : 32 }
    fp.numerical2physical( params, physical)
    physicals.append(physical)

Invert for given R
Invert for given R
Invert for given R
Invert for given R
Invert for given R
Invert for given R


In [6]:
data=list()
columns=[*physicals[0].keys(), *fp.quantities() ]
for physical in physicals:
    data.append( fp.parameters2quantities( physical, columns))
df = pd.DataFrame( data, columns=columns)

In [54]:
def orderToString(x): 
    if np.isnan(x) : return 'n/a'
    return'%.2f'% x
def errorToString(x):
    return '%.2e' % x

In [56]:

for s in ["n_0", "R", "m_i", "rho_s"]:
    print( s, "\t\t", fp.parameters2quantity( physical, s))
show = ["resistivity", "B_0", "T_e", "omega_0_inv"]
df_str = df[show]
df_str.columns = ["$\eta$","$B_0/$T", "$T_e/$eV", "$\Omega_0^{-1}/$s"]

for header in ["$\eta$", "$\Omega_0^{-1}/$s"] :  
    df_str[header] = df_str[header].apply( errorToString)
for header in ["$B_0/$T", "$T_e/$eV"] :  
    df_str[header] = df_str[header].apply( orderToString)
df_str.set_index("$\eta$", inplace=True)

filename = "parameters.tex"
with open(filename, 'wb') as f:
    f.write(bytes(df_str.style.to_latex(
        column_format='rrrr', hrules=True),'UTF-8'))
    

df_str

n_0 		 1.0365292394443688
R 		 0.545
m_i 		 3.343583719e-27
rho_s 		 0.0010000000002649348


<ipython-input-56-b863b5165469>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_str[header] = df_str[header].apply( errorToString)
<ipython-input-56-b863b5165469>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_str[header] = df_str[header].apply( orderToString)


,$B_0/$T,$T_e/$eV,$\Omega_0^{-1}/$s
$\eta$,,,
1.00e-06,1.27,77.76,1.64e-08
3.00e-06,0.97,44.90,2.16e-08
1.00e-05,0.72,24.59,2.91e-08
3.00e-05,0.54,14.20,3.83e-08
1.00e-04,0.40,7.78,5.18e-08
3.00e-04,0.31,4.49,6.82e-08


## 